Select the gene of interest, the species you want to compare, and the position of the residue.

In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import appmode
import pandas
from ipywidgets import FloatSlider
import os
from ipywidgets import Button, Layout
from ipywidgets import HBox, Label, VBox
import tabulate


In [2]:
list_genes=pandas.read_csv("Available_genes.tab", sep="\t", header=None)
list_genes.columns=['Gene', 'File']
list_genes=list_genes.set_index('Gene').to_dict('index')


In [7]:
list_genes

{'Spike Protein': {'File': 'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/S/S_coordinates.tab'},
 'Membrane Protein': {'File': 'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/M/M_coordinates_conversion.tab'},
 'Nucleocapside Protein': {'File': 'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/N/N_coordinates_conversion.tab'},
 'RNA dependent RNA polymerase (RdRp)': {'File': 'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/RdRp/RdRp_coordinates_conversion.tab'},
 '3Clike protease': {'File': 'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/3CLpro/3CLpro_coordinates_conversion.tab'},
 'ORF3': {'File': 'https://raw.githubusercontent.com/galaxyproject/SARS-CoV-2/master/genomics/7-VariantsDescription/ORF3/ORF3_coordinates.tab'}}

In [3]:
dic_tables={}
for i in list_genes.keys():
    dic_tables[i]=pandas.read_csv(list_genes[i]['File'], sep="\t")

In [5]:
dic_tables.keys()

dict_keys(['Spike Protein', 'Membrane Protein', 'Nucleocapside Protein', 'RNA dependent RNA polymerase (RdRp)', '3Clike protease', 'ORF3'])

In [6]:
dic_tables['Spike Protein'].head()

,AA,NC_045512,Alignment,FJ647223,JX504050,NC_038294,NC_004718,FJ647218,MN996527,NC_028806,AY714217,KF367457,NC_038861,NC_002306,MG772934,AY463059,MN996528,DQ022305
0,M,1,8,2,0,5,1,2,1,7,1,2,4,8,2,1,1,5
1,F,2,9,3,0,6,2,3,2,8,2,3,5,9,3,2,2,6
2,V,3,11,5,3,7,3,4,3,9,3,4,6,10,4,3,3,7
3,F,4,12,6,4,8,4,5,4,10,4,5,7,11,5,4,4,8
4,L,5,13,7,5,9,5,6,5,11,5,6,8,12,6,5,5,9


## Single Residue

In [ ]:

directory = widgets.Dropdown(options=[""]+list(list_genes.keys()),layout=Layout(width='50%', height='30px', align_content='center'))
colonnes = widgets.Dropdown(disabled=False,layout=Layout(width='50%', height='30px', align_content='center'))
position = widgets.IntText(disabled=False,layout=Layout(width='80%', height='30px',align_content='center'))

def update_columns(*args):
    table=dic_tables[directory.value]
    tmp=list(table.columns)
    tmp.remove('AA')
    colonnes.options = tmp

def update_pos(*args):
    table=dic_tables[directory.value]
    position.max=int(table[colonnes.value].max())
    
            
directory.observe(update_columns, 'value')
colonnes.observe(update_pos, 'value')



def show_position(Gene, Species,Position):
    if Gene!="":
        table=dic_tables[directory.value]
        if len(table[table[Species]==Position])==0:
            print('No results')
        else:
            return table[table[Species]==Position]

        


_ = interact_manual(show_position, Gene=directory, Species=colonnes, Position=position)





## Region

In [ ]:

directory_reg = widgets.Dropdown(options=[""]+list(list_genes.keys()),layout=Layout(width='50%', height='30px', align_content='center'))
colonnes_reg = widgets.Dropdown(disabled=False,layout=Layout(width='50%', height='30px', align_content='center'))
start = widgets.IntText(disabled=False,layout=Layout(width='30%', height='30px',align_content='center'))
end = widgets.IntText(disabled=False,layout=Layout(width='30%', height='30px',align_content='center'))

def update_columns_reg(*args):
    table=dic_tables[directory.value]
    tmp=list(table.columns)
    tmp.remove('AA')
    colonnes_reg.options = tmp

def update_start(*args):
    table=dic_tables[directory.value]
    start.max=int(table[colonnes_reg.value].max())
    start.value=0

def update_max_start(*args):
    start.max=end.value

def update_min_end(*args):
    end.min=start.value


def update_end(*args):
    table=dic_tables[directory.value]
    end.max=int(table[colonnes_reg.value].max())
    end.value=int(table[colonnes_reg.value].max())

directory_reg.observe(update_columns_reg, 'value')
colonnes_reg.observe(update_start, 'value')
colonnes_reg.observe(update_end, 'value')
end.observe(update_max_start, 'value')
start.observe(update_min_end, 'value')



def show_position_reg(Gene, Species,Start,End):
    if Gene!="":
        table=dic_tables[directory.value]
        if len(table[(table[Species]>=Start) & (table[Species]<=End)])==0:
            print('No results')
        else:
            return table[(table[Species]>=Start) & (table[Species]<=End)]

_ = interact(show_position_reg, Gene=directory_reg, Species=colonnes_reg, Start=start,End=end )

HBox([Label('Position'), start, end])



In [ ]:

directory_reg = widgets.Dropdown(options=[""]+list(list_genes.keys()),layout=Layout(width='30%', height='30px', align_content='center'))
colonnes_reg = widgets.Dropdown(disabled=False,layout=Layout(width='30%', height='30px', align_content='center'))
start = widgets.IntText(disabled=False,layout=Layout(width='30%', height='30px',align_content='center'))
end = widgets.IntText(disabled=False,layout=Layout(width='30%', height='30px',align_content='center'))
topbox=HBox([Label(' Table'), directory_reg, colonnes_reg])
botbox=HBox([Label('Position'), start, end])
okbutton=Button(description='Submit') 
allboxes=VBox([Label(''), topbox, botbox,okbutton])


def update_columns_reg(*args):
    table=dic_tables[directory.value]
    tmp=list(table.columns)
    tmp.remove('AA')
    colonnes_reg.options = tmp

def update_start(*args):
    table=dic_tables[directory.value]
    start.max=int(table[colonnes_reg.value].max())
    start.value=0

def update_max_start(*args):
    start.max=end.value

def update_min_end(*args):
    end.min=start.value


def update_end(*args):
    table=dic_tables[directory.value]
    end.max=int(table[colonnes_reg.value].max())
    end.value=int(table[colonnes_reg.value].max())



def show_clicked(okbutton):
    display(show_position_reg(Gene=directory_reg.value, Species=colonnes_reg.value, Start=start.value,End=end.value))


directory_reg.observe(update_columns_reg, 'value')
colonnes_reg.observe(update_start, 'value')
colonnes_reg.observe(update_end, 'value')
end.observe(update_max_start, 'value')
start.observe(update_min_end, 'value')
okbutton.on_click(show_clicked)


def show_position_reg(Gene, Species,Start,End):
    if Gene!="":
        table=dic_tables[Gene]
        if len(table[(table[Species]>=Start) & (table[Species]<=End)])==0:
            print('No results')
        else:
            return table[(table[Species]>=Start) & (table[Species]<=End)]

        

out = widgets.interactive_output(show_position_reg, {'Gene': directory_reg, 'Species': colonnes_reg, 'Start':start,'End':end})

display(allboxes, out)


In [ ]:
show_position_reg(Gene=directory_reg.value, Species=colonnes_reg.value, Start=start.value,End=end.value)

In [244]:
class foo(object):

    def __init__(self, file1, file2):
        self.filenames = [file1, file2]
        self.file_dict = {
                file1:pd.DataFrame(np.arange(5)),
                file2:pd.DataFrame(np.arange(10,15))
            }

    def _create_widgets(self):
        self.sel_file = widgets.SelectMultiple(description='Files',
                                               options=self.filenames,
                                               value=[self.filenames[0]],
                                              )
        self.button = widgets.Button(description="OK")
        self.button.on_click(self._on_button_clicked)

    def _on_button_clicked(self, change):
        self.out.clear_output()
        self.df_objects = {}
        with self.out:
            for f in self.sel_file.value:
                print(f)
                display(self.file_dict[f])
                self.df_objects[f] = self.file_dict[f]

    def display_widgets(self):
        self._create_widgets()
        self.out = widgets.Output()  # this is the output widget in which the df is displayed
        display(widgets.VBox(
                            [
                                self.sel_file,
                                self.button,
                                self.out
                            ]
                        )
               )

    def get_df_objects(self):
        return self.df_objects


In [ ]:
class foo(object):


directory_reg = widgets.Dropdown(options=[""]+list(list_genes.keys()),layout=Layout(width='30%', height='30px', align_content='center'))
colonnes_reg = widgets.Dropdown(disabled=False,layout=Layout(width='30%', height='30px', align_content='center'))
start = widgets.IntText(disabled=False,layout=Layout(width='30%', height='30px',align_content='center'))
end = widgets.IntText(disabled=False,layout=Layout(width='30%', height='30px',align_content='center'))
topbox=HBox([Label(' Table'), directory_reg, colonnes_reg])
botbox=HBox([Label('Position'), start, end])
okbutton=Button(description='Submit') 
allboxes=VBox([Label(''), topbox, botbox,okbutton])


def update_columns_reg(*args):
    table=dic_tables[directory.value]
    tmp=list(table.columns)
    tmp.remove('AA')
    colonnes_reg.options = tmp

def update_start(*args):
    table=dic_tables[directory.value]
    start.max=int(table[colonnes_reg.value].max())
    start.value=0

def update_max_start(*args):
    start.max=end.value

def update_min_end(*args):
    end.min=start.value


def update_end(*args):
    table=dic_tables[directory.value]
    end.max=int(table[colonnes_reg.value].max())
    end.value=int(table[colonnes_reg.value].max())